# タスク 3: RAG 評価 (RAGAS) フレームワークを使用し、Amazon Bedrock ナレッジベースを使って質問応答アプリケーションを構築および評価する

このタスクでは、LangChain の **AmazonKnowledgeBasesRetriever** クラス、Chains、および応答を評価するための RAGAS フレームワークを使用して、質問応答アプリケーションの構築と評価を行います。ここでは、ナレッジベースにクエリを実行して、類似検索に基づいて必要な数のドキュメントチャンクを取得します。次に、クエリと共にドキュメントチャンクをコンテキストとして指定して、テキスト生成 LLM にプロンプトを出します。次に、「faithfulness (忠実度)」、「answer_relevancy (回答の関連性)」、「context_recall (コンテキスト再現率)」、「context_precision (コンテキスト精度)」、「context_entity_recall (コンテキストエンティティ再現率)」、「answer_similarity (回答類似性)」、「answer_correctness (回答の正確性)」、「harmfulness (有害性)」、「maliciousness (悪意の有無)」、「coherence (一貫性)」、「correctness (正しさ)」、「conciseness (簡潔さ)」という評価メトリクスを使用して応答を評価します。

<i aria-hidden="true" class="fas fa-info-circle" style="color:#007FAA"></i> **詳細:** Ragas フレームワークで使用されるさまざまなメトリクスに関する追加情報については、**[Metrics](https://docs.ragas.io/en/latest/concepts/metrics/index.html)** を参照してください。

### パターン

検索拡張生成 (RAG) パターンを使用してソリューションを実装します。RAG は、言語モデルの外部からデータを取得し、関連する取得データをコンテキストに追加することでプロンプトを拡張します。このタスクでは、ラボのプロビジョニング中に既に作成済みのナレッジベースを使用して、クエリへの応答を作成します。

#### 評価

- RAGAS を利用して次のメトリクスを評価します。
  - **Faithfulness (忠実度):** 生成された回答の事実の一貫性を、与えられたコンテキストと照らし合わせて測定します。これは、回答と、取得したコンテキストから計算されます。回答は (0,1) の範囲で計測されます。高いほど忠実度が優れています。
  - **Answer Relevance (回答の関連性):** このメトリクスは、生成された回答が特定のプロンプトにどの程度関連しているかを評価することに重点を置いています。不完全または冗長な情報を含む回答には低いスコアが割り当てられ、スコアが高いほど関連性が高くなります。このメトリクスは、質問、コンテキスト、および回答を使用して計算されます。実際のところ、スコアの範囲はほとんどの場合 0～1 ですが、コサイン類似度が -1～1 の範囲にあるため、これは数学的に保証されているわけではないことに注意してください。
  - **Context Precision (コンテキスト精度):** これは、コンテキストに存在する Ground Truth の関連項目すべてが上位にランクされているかどうかを評価するメトリクスです。理想的には、関連性の高いすべてのチャンクがランク上位に表示される必要があります。このメトリクスは、質問、ground_truth、コンテキストを使用して計算され、値は 0～1 の範囲です。スコアが高いほど精度が高くなります。
  - **Context Recall (コンテキスト再現率):** このメトリクスは、取得したコンテキストが注釈付きの回答とどの程度一致しているかを測定し、Ground Truth となる真実として扱います。Ground Truth および取得したコンテキストに基づいて計算され、値の範囲は 0～1 です。値が大きいほどパフォーマンスが高くなります。
  - **Context entities recall (コンテキストエンティティ再現率):** このメトリクスは、ground_truths のみに存在するエンティティの数と、ground_truths とコンテキストの両方に存在するエンティティの数との割合に基づいて、取得したコンテキストの再現率を計算します。簡単に言えば、ground_truths から再現されたエンティティの割合を示す指標です。このメトリクスは、観光ヘルプデスクや歴史 QA など、事実に基づくユースケースに役立ちます。このメトリクスは、ground_truths に存在するエンティティとの比較に基づいて、エンティティの検索メカニズムを評価するのに役立ちます。エンティティが重要な場合は、そうしたエンティティを対象とするコンテキストが必要だからです。
  - **Answer Semantic Similarity (回答セマンティクス類似性):** 回答セマンティック類似性の概念は、生成された回答と Ground Truth との間の意味的類似性の評価に関係します。この評価は Ground Truth と回答に基づいており、値は 0～1 の範囲に収まります。スコアが高いほど、生成された回答と Ground Truth との整合性が優れています。
  - **Answer Correctness (回答の正確性):** 回答の正確性の評価には、生成された回答の正確さを、Ground Truth と比較して測定することが含まれます。この評価は Ground Truth と回答に基づいており、スコアは 0～1 です。スコアが高いほど、生成された回答と Ground Truth との間により近い一致を示し、正確性が高いことを示します。回答の正確性には、生成された回答と Ground Truth との間の意味的類似性と、事実上の類似性という 2 つの重要な要素が含まれます。これらの要素は加重スキームを用いて組み合わされ、回答の正解スコアが算出されます。また、必要に応じて「しきい値」値を使用して結果のスコアを丸めてバイナリにすることもできます。
  - **Aspect Critique (アスペクト批評):** これは、無害性や正確性など、事前に定義された要素に基づいて提出物を評価することを目的としています。アスペクト批評の出力はバイナリ形式で、提出物が定義されたアスペクトと一致しているかどうかを示します。この評価は、「回答」を入力として使用して実行されます。

このタスクでは、AnyCompany の 10K 財務報告 (合成的に生成されたデータセット) をテキストコーパスとして使用して質問応答を行います。このデータは既に Amazon Bedrock のナレッジベースに取り込まれています。

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **注:** 特定のユースケースでは、ドメイントピックの異なるさまざまなファイルを同期し、同じ方法でこのノートブックにクエリを実行して、Retrieve API を使用してナレッジベースからのモデル応答を評価できます。

<i aria-hidden="true" class="fas fa-exclamation-circle" style="color:#7C5AED"></i> **注意:** [**Run**] メニューの [**Run All Cells**] オプションを使用するよりも、各コードセルを個別に実行することをお勧めします。すべてのセルをまとめて実行すると、カーネルがクラッシュしたり再起動したりするなど、予期しない動作が発生することがあります。セルを 1 つずつ実行することで、実行フローをより適切に制御し、潜在的なエラーを早期に検出し、コードを意図したとおりに実行することができます。

## タスク 3.1: 環境を設定する

このノートブックを実行するには、依存関係である LangChain と RAGAS、そして更新された boto3、botocore パッケージをインストールする必要があります。

以下に示す手順に従って、必要なパッケージを設定します。

- 基盤モデルを呼び出す **bedrock-runtime** を作成するために必要なライブラリをインポートします。
- LangChain 関連ライブラリをインポートします。
- 大規模言語モデルとして Bedrock モデル **amazon.titan-text-premier-v1:0** を初期化し、RAG パターンを使用してクエリ補完を実行します。
- 大規模言語モデルとして Bedrock モデル **amazon.nova-lite-v1:0** を初期化し、RAG 評価を実行します。
- 大規模言語埋め込みモデルとして Bedrock モデル **amazon.titan-embed-text-v2:0** を初期化し、RAG 評価用の埋め込みを作成します。これは、ナレッジベースの作成に使用されたのと同じ埋め込みモデルです。
- ナレッジベースと統合された LangChain リトリーバーを初期化します。
- ノートブックの後半で、LLM とリトリーバーをチェーンとしてまとめて、質問応答アプリケーションを作成します。

1. 次のコードセルを実行して、Amazon Bedrock の既存のナレッジベース ID を確認します。

In [ ]:
import botocore
import boto3

session = boto3.Session()
bedrock_client = session.client('bedrock-agent')

try:
    response = bedrock_client.list_knowledge_bases(
        maxResults=1  # We only need to retrieve the first Knowledge Base
    )
    knowledge_base_summaries = response.get('knowledgeBaseSummaries', [])

    if knowledge_base_summaries:
        kb_id = knowledge_base_summaries[0]['knowledgeBaseId']
        print(f"Knowledge Base ID: {kb_id}")
    else:
        print("No Knowledge Base summaries found.")
        
except botocore.exceptions.ClientError as e:
    print(f"Error: {e}")

2. 次のコードセルを実行して依存関係をインストールします。

In [ ]:
import boto3
import pprint

from langchain_aws import ChatBedrock
from langchain_aws import BedrockEmbeddings
from langchain_community.retrievers import AmazonKnowledgeBasesRetriever

pp = pprint.PrettyPrinter(indent=2)

bedrock_client = boto3.client('bedrock-runtime')

llm_for_text_generation = ChatBedrock(model_id="amazon.nova-lite-v1:0", client=bedrock_client)
llm_for_evaluation = ChatBedrock(model_id="amazon.nova-lite-v1:0", client=bedrock_client)

bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0",client=bedrock_client)

## タスク 3.2: LangChain から **AmazonKnowledgeBasesRetriever** オブジェクトを作成する

このタスクでは、LangChain から **AmazonKnowledgeBasesRetriever** オブジェクトを作成してナレッジベースを検索し、関連する結果を返します。これにより、セマンティクス検索結果に基づいてカスタムワークフローをより詳細に構築できるようになります。

3. 次のコードセルを実行して **AmazonKnowledgeBasesRetriever** オブジェクトを作成します。

In [2]:
retriever = AmazonKnowledgeBasesRetriever(
        knowledge_base_id=kb_id,
        retrieval_config={"vectorSearchConfiguration": {"numberOfResults": 5}},
        # endpoint_url=endpoint_url,
        # region_name="us-east-1",
        # credentials_profile_name="<profile_name>",
    )

## タスク 3.3: RetrievalQA チェーンを使用してモデルを呼び出し、応答を生成する 

このタスクでは、次の情報を使用してモデルを呼び出し、応答を視覚化します。

質問 =

```
AnyCompany の財務に関するいくつかのリスクのリストを、説明なしの番号付きリストで提供します。"
```

Ground Truth 回答 = 

```
1. 商品価格
2. 外国為替レート 
3. 株価
4. 信用リスク
5. 流動性リスク
...
...
```

4. 次のコードセルを実行して、コンテキストと質問を変数とするプロンプトを作成します。

In [ ]:
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE = """
Human: あなたはファイナンシャルアドバイザーのAIシステムであり、可能な限り事実に基づいた統計情報を用いて質問に答えます。
以下の情報を用いて、<question>タグで囲まれた質問に簡潔に回答してください。
わからない部分がある場合は、その部分は「わからない」とだけ伝え、わざわざ答えを作ろうとしないでください。
<context>
{context}
</context>

<question>
{question}
</question>

回答は具体的なものでなければならず、可能な場合は統計や数字を使用する必要があります。

Assistant:"""
prompt = PromptTemplate(template=PROMPT_TEMPLATE, 
                               input_variables=["context","question"])

5. 次のコードセルを実行して、定義済みのクエリを使用してモデルを呼び出し、結果を出力します。

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs): #concatenate the text from the page_content field in the output from retriever.invoke
    return "\n\n".join(doc.page_content for doc in docs)

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm_for_text_generation
    | StrOutputParser()
)

query = "AnyCompanyの財務リスク10項目を番号付きリストで提供してください。説明は含めないでください。"

response=chain.invoke(query)
print(response)

## タスク 3.4: 評価データを準備する

RAGAS は参照不要の評価フレームワークを目指しているため、評価データセットの準備は最小限で済みます。このタスクでは、次に示すように、**question** と **ground_truths** のペアを用意し、そこから推論によって以下のように残りの情報を準備します。

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **注:** **context_recall** メトリクスを調べる必要がない場合は、**ground_truths** 情報を提供する必要はありません。このタスクで準備する必要があるのは、**question** だけとなります。

5. 次のコードセルを実行して、評価のための **question** と **ground_truths** のペアを準備します。この処理は、スロットリングが発生した場合には再試行しながら、数分間実行される可能性があります。

In [ ]:
from datasets import Dataset
import time
import random

# Define questions and ground truths for RAGAS evaluation
questions = [
    "2021 年に AnyCompany Financial の営業活動による純現金が増加した主な理由は何ですか?",
    "AnyCompany Financial の投資活動における純現金使用が最も高かったのはどの年ですか。主な理由は何ですか。",
    "2021 年の AnyCompany Financial の財務活動によるキャッシュインフローの主な源泉は何でしたか?",
    "2020 年から 2021 年にかけての AnyCompany Financial の現金および現金同等物の前年比変化率を計算します。",
    "提供された情報に基づいて、AnyCompany Financial の全体的な財務状況と成長の見通しについてどのようなことが推測できますか?"
]

ground_truth = [
    "営業活動による純キャッシュ・フローの増加は、主に純利益の増加と営業資産および負債の好ましい変動によるものです。",
    "AnyCompany Financialの2021年の投資活動における純現金使用額は3億6,000万ドルで、2020年の2億9,000万ドル、2019年の2億4,000万ドルと比較して過去最高となりました。主な理由は、有形固定資産および市場性ある有価証券の購入の増加です。",
    "2021 年の AnyCompany Financial の財務活動によるキャッシュインフローの主な源泉は、普通株式および長期債務の発行による収入の増加でした。",
    "2020年から2021年にかけての現金および現金同等物の前年比変化率を計算する: \
    2020年の現金および現金同等物：3億5000万ドル \
    2021年の現金および現金同等物：4億8000万ドル \
    パーセンテージの変化 = (2021 value - 2020 value) / 2020 value * 100 \
    = ($480 million - $350 million) / $350 million * 100 \
    = 37.14% increase",
    "提供された情報に基づくと、AnyCompany Financialは健全な財務状況にあり、良好な成長見通しを有していると考えられます。同社は営業活動による純キャッシュフローが増加しており、高い収益性と運転資本の効率的な運用を示しています。AnyCompany Financialは有形固定資産や有価証券などの長期資産への投資を行っており、これは将来の成長と拡大に向けた計画を示唆しています。同社は普通株式の発行と長期債務の発行を通じて成長資金を調達しており、これは投資家と貸し手からの信頼を示しています。全体として、AnyCompany Financialの過去3年間の現金および現金同等物の着実な増加は、将来の成長と投資機会のための強固な基盤となっています。"
]

def get_model_response(query, chain, retriever, max_retries=5, wait_time=15):
    """Get response from the model with fixed wait time between retries"""
    for attempt in range(max_retries):
        try:
            # Configure Nova Lite with increased tokens
            nova_config = {
                "schemaVersion": "messages-v1",
                "messages": [{
                    "role": "user",
                    "content": [{"text": query}]
                }],
                "inferenceConfig": {
                    "maxTokens": 2048,
                    "temperature": 0.5,
                    "topP": 0.9,
                    "topK": 20
                }
            }
            
            # Try to invoke with config override
            try:
                answer = chain.invoke(
                    query,
                    config_override={"model_kwargs": nova_config}
                )
            except AttributeError:
                # If config_override doesn't work, try direct invocation
                answer = chain.invoke(query)
            
            context = [docs.page_content for docs in retriever.invoke(query)]
            print(f"Successfully processed query on attempt {attempt + 1}")
            return answer, context
            
        except Exception as e:
            if attempt == max_retries - 1:
                print(f"Failed after {max_retries} attempts for query: {query[:50]}...")
                print(f"Error: {str(e)}")
                return None, None
            print(f"Attempt {attempt + 1} failed, waiting {wait_time} seconds before retry...")
            time.sleep(wait_time)

# Process questions one at a time with fixed delay
answers = []
contexts = []

print("Starting to process questions...")
for i, query in enumerate(questions, 1):
    print(f"\nProcessing question {i}/{len(questions)}")
    print(f"Query: {query[:100]}...")
    
    answer, context = get_model_response(query, chain, retriever)
    if answer is not None:
        answers.append(answer)
        contexts.append(context)
        print(f"Successfully processed question {i}")
    else:
        print(f"Failed to process question {i}")
    
    #if i < len(questions):
    #    print(f"Waiting 60 seconds before next question...")
    #    time.sleep(60)

# Create dataset for RAGAS evaluation
data = {
    "question": questions[:len(answers)],
    "ground_truth": ground_truth[:len(answers)],
    "answer": answers,
    "contexts": contexts
}

# Convert to dataset
dataset = Dataset.from_dict(data)

# Print dataset information
print("\nDataset Creation Summary:")
print(f"Total questions processed: {len(dataset)} out of {len(questions)}")
print(f"Columns available: {dataset.column_names}")

# Print sample entry
if len(dataset) > 0:
    print("\nSample Entry (First Question):")
    print(f"Question: {dataset[0]['question']}")
    print(f"Ground Truth: {dataset[0]['ground_truth']}")
    print(f"Model Answer: {dataset[0]['answer']}")
else:
    print("\nNo entries were successfully processed into the dataset.")


6. 次のコードセルを実行して、LLM からの回答と、一連の質問評価のための Ground Truth を確認します。

In [ ]:
i=0
for answer in answers:
    i=i+1
    print(str(i)+').'+questions[i-1]+'\n')
    print("LLM:" +answer+'\n')
    print ("Ground truth: "+ ground_truth[i-1]+'\n')

## タスク 3.5: RAG アプリケーションを評価する

このタスクでは、使用したいすべてのメトリクスを **ragas.metrics** からインポートします。次に、**evaluate()** 関数を使用して、関連するメトリクスと準備したデータセットをそのまま渡します。

7. 次のコードセルを実行して **ragas.metrics** からすべてのメトリクスをインポートし、**evaluate()** 関数を使用します。

In [ ]:
import warnings
import logging

warnings.filterwarnings('ignore')   # ignore warnings related to pydantic v1 to v2 migration
logging.getLogger('root').setLevel(logging.CRITICAL)

from datasets import Dataset
if not hasattr(Dataset, 'from_list'):
    def from_list_compatibility(data_list):
        if isinstance(data_list, list) and len(data_list) > 0 and isinstance(data_list[0], dict):
            keys = data_list[0].keys()
            data_dict = {key: [item[key] for item in data_list] for key in keys}
            return Dataset.from_dict(data_dict)
        return Dataset.from_dict({})
    Dataset.from_list = staticmethod(from_list_compatibility)

from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    context_entity_recall,
    answer_similarity,
    answer_correctness
)

from ragas.metrics.critique import (
harmfulness, 
maliciousness, 
coherence, 
correctness, 
conciseness
)

#specify the metrics here
metrics = [
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
        context_entity_recall,
        answer_similarity,
        answer_correctness,
        harmfulness, 
        maliciousness, 
        coherence, 
        correctness, 
        conciseness
    ]

try:
    result = evaluate(
        dataset=dataset,
        metrics=metrics,
        llm=llm_for_evaluation,
        embeddings=bedrock_embeddings,
    )
    df = result.to_pandas()
except Exception as e:
    # Handle any exceptions that occur during the evaluation
    print(f"An error occurred: {e}")

<i aria-hidden="true" class="fas fa-exclamation-circle" style="color:#7C5AED"></i> **注意:** 上の出力の警告は無視しても問題ありません。次に進む前に、評価が 100% 完了していることを確認してください。このステップは完了するまでに約 7～10 分かかります。

8. 以下のコードセルを実行して、結果の RAGAS スコアを確認します。

In [ ]:
import pandas as pd
pd.options.display.max_colwidth = 10
df.style.set_sticky(axis="columns")

9. 以下のコードセルを実行して、結果の RAGAS スコアを Excel 形式でエクスポートします。

In [7]:
df.style.to_excel('styled.xlsx', engine='openpyxl')

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **注:** 上のコードセルを正常に実行すると、左側のナビゲーションペインの **ja_jp** フォルダの下に **styled.xlsx** という名前の確認用のファイルが表示されるはずです。ファイルを開くのに時間がかかりすぎる場合は、ファイルを右クリックして [**Open in New Browser Tab**] を選択します。

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **注:** 上のスコアは、RAG アプリケーションのパフォーマンスを相対的に評価するものであり、単独のスコアとしてではなく、慎重に使用する必要があることに注意してください。また、評価に使用した質問と回答のペアは 5 つだけであることにも注意してください。ベストプラクティスとして、モデルを評価する際は、ドキュメントのさまざまな要素をカバーするのに十分なデータを使用する必要があります。

スコアに基づいて RAG ワークフローの他のコンポーネントを確認すると、スコアをさらに最適化できます。推奨されるオプションとしては、チャンキング戦略やプロンプト命令を見直すこと、コンテキスト追加のために numberOfResults を大きくすることなどがあります。

<i aria-hidden="true" class="far fa-thumbs-up" style="color:#008296"></i> **タスクの完了:** このノートブックを完了しました。ラボの次の部分に進むために、以下を実行してください。

- このノートブックファイルを閉じます。
- ラボセッションに戻り、タスク 4 に進みます。